In [1]:
import json, os
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from string import punctuation
from nltk.corpus import stopwords
from collections import Counter
from itertools import chain, combinations
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [5]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [6]:
data.shape

(1987, 5)

In [7]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp + fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec + rec) == 0:
            f1 = 0
        else:
            f1 = (2 * (prec * rec)) / (prec + rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

## Токенизация, удаление стоп-слов и нормализация.

#### 1) Сравним результаты mystem и pymorphy:

In [8]:
punct = punctuation + '«»—…“”*№–'
stops = set(stopwords.words('russian'))
mystem = Mystem()
morph = MorphAnalyzer()

def normalize(text, method='mystem'):
    
    words = [word.strip(punct) for word in text.lower().split()]
    if method == 'pymorphy':
        words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    else:
        words = [mystem.lemmatize(word)[0] for word in words if word and word not in stops]

    return words

In [9]:
data['content_norm'] = data['content'].apply(normalize)

In [10]:
data['title_norm'] = data['title'].apply(normalize)

In [11]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


В исходном варианте (c pymorphy) качество на этом шаге ниже:
* Precision -  0.1
* Recall -  0.21
* F1 -  0.13
* Jaccard -  0.08

Поэтому в дальнейшем будет использоваться mystem.

#### 2) Добавим в список стоп-слов сокращения от единиц измерений, названий валют и т.п.:
Найдем наиболее частотные сокращения, которые анализируются как существительные:

In [12]:
d = Counter()

for elem in chain.from_iterable(data['content_norm']):
    if len(elem) < 5:
        d[elem] += 1

In [13]:
#просмотрим соответствующий список (полностью)
d.most_common()

[('год', 11167),
 ('это', 9217),
 ('свой', 4722),
 ('один', 3026),
 ('мочь', 2204),
 ('наш', 2182),
 ('сша', 2089),
 ('дело', 1991),
 ('мир', 1952),
 ('тот', 1894),
 ('быть', 1857),
 ('этот', 1604),
 ('день', 1571),
 ('сила', 1549),
 ('идти', 1423),
 ('рф', 1288),
 ('сам', 1240),
 ('два', 1206),
 ('тыс', 1173),
  ...]

Добавим в стоп-слова:

In [14]:
stops2 = stops.union({'тыс', 'млрд', 'нга', 'м', 'т', 'долл', 'км', 'год',
                 'бла', 'куб', 'евро', 'см', 'г', 'кг', 'трлн', 'др', 'кв', 'га'})

In [15]:
mystem = Mystem()

def normalize2(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    nouns = []
    for word in words:
        if word and word not in stops2:
            analysis = mystem.analyze(word)[0].get('analysis')
            if analysis and analysis[0]['gr'].split(',')[0] == 'S':
                nouns.append(analysis[0]['lex'])
                
    return nouns

In [16]:
data['content_norm_2'] = data['content'].apply(normalize2)

In [17]:
evaluate(data['keywords'], data['content_norm_2'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


В исходном варианте качество на этом шаге:
* Precision -  0.12
* Recall -  0.24
* F1 -  0.15
* Jaccard -  0.09

#### 3) Проанализируем заголовки вместе с основным текстом:
Это может помочь, поскольку, по результатам, полученным на семинаре, нахождение ключевых слов только из заголовков показывает результаты не намного хуже, чем нахождение ключевых слов из целых текстов.

In [18]:
data['norm'] = (data['content'] + data['title']).apply(normalize2)

In [19]:
evaluate(data['keywords'], data['norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.14
Recall -  0.27
F1 -  0.17
Jaccard -  0.1


Улучшилось ```precision``` и ```recall``` на 0.1.

Воспользуемся TfidfVectorizer.

In [20]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, lowercase=False, tokenizer=lambda x: x)

In [21]:
tfidf.fit(data['content_norm'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7f11dcc36730>, use_idf=True,
        vocabulary=None)

In [22]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [23]:
texts_vectors = tfidf.transform(data['norm'])

#### топ-10:

In [24]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [25]:
evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.27
F1 -  0.18
Jaccard -  0.1


В исходном варианте качество на этом шаге:
* Precision -  0.13
* Recall -  0.24
* F1 -  0.16
* Jaccard -  0.09

#### 4) Возьмем топ-5 вместо топ-10.

In [26]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [27]:
evaluate(data['keywords'], keywords)

Precision -  0.2
Recall -  0.2
F1 -  0.19
Jaccard -  0.12


Выросло ```F1``` и ```precision``` и ```jaccard```, но, к сожалению, уменьшился ```recall``` (в сравнении с топ-10).

## Попробуем графы!

In [28]:
import networkx as nx

In [29]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

#### топ-10:

In [30]:
keyword_nx = data['norm'].apply(lambda x: some_centrality_measure(x, 10, 10))
evaluate(data['keywords'], keyword_nx)

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


В исходном варианте качество на этом шаге:
* Precision -  0.12
* Recall -  0.24
* F1 -  0.16
* Jaccard -  0.09

#### 4) Возьмем топ-5 вместо топ-10: 

In [31]:
keyword_nx = data['norm'].apply(lambda x: some_centrality_measure(x, 10, 5))
evaluate(data['keywords'], keyword_nx)

Precision -  0.19
Recall -  0.19
F1 -  0.18
Jaccard -  0.11


Также значения всех метрик, кроме ```recall```, увеличились (в сравнении с топ-10).